# SIMPLE TUTORIAL: USING DATA SUITE ON SYNTHETIC DATA

## Imports

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

import numpy as np
import pandas as pd
import wandb
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from src.data.data_loader import load_adult_data
from src.models.benchmarks import comparison_methods
from src.models.conformal import conformal_class
from src.models.representation import compute_representation
from src.utils.data_utils import (
    covariance_comparison,
    get_suspect_features,
    read_from_file,
    write_to_file,
)
from src.utils.helpers import inlier_outlier_dicts, sort_ci_vals

model_ids = {}
artifact_path = "artifacts"



In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Generate synthetic data

In [4]:

# examples to try
props = [0.1, 0.25, 0.5, 0.75]
dists = ["normal", "beta", "gamma", "weibull"]
noise_vars = [1, 2, 5, 10]
copula_count_samples = [1000]

# set defaults
prop = props[0]
dist = dists[0]
noise_variance = noise_vars[0]
copula_n_samples = 1000


n_synthetic = 1000
train_prop = 1
rep_type = "pca"

wandb_dict = {}


from src.data.data_loader import load_synthetic_data

(
    train,
    test,
    orig_test,
    noise_bool,
    noise_matrix,
    noise_idx,
) = load_synthetic_data(
    n_synthetic=n_synthetic,
    mean=0,
    noise_variance=noise_variance,
    dim="small",
    prop=prop,
    dist=dist,
)

## Initialize & fit Data Suite

In [5]:
from data_suite import Data_SUITE
ds = Data_SUITE(copula_type='vine', 
                n_copula_samples=1000, 
                representer='pca', 
                rep_dim=None)

ds.fit(train)

INFO:root:Vine...
INFO:copulas.multivariate.vine:Fitting VineCopula("direct")
INFO:root:Copula Samples = 1000


## Predict on test data

In [6]:
conformal_dict, suspect_features = ds.predict(test)

INFO:root:Running analysis for feature = 0
INFO:root:Running analysis for feature = 1
INFO:root:Running analysis for feature = 2


## Process conformal dict

In [7]:
from src.utils.helpers import *
proportion=0.1

inliers_dict, outliers_dict = inlier_outlier_dicts(
            conformal_dict, suspect_features
        )

small_ci_ids, large_ci_ids, df_sorted = sort_cis_synth(
    conformal_dict, inliers_dict, suspect_features=[0], proportion=proportion
)

/Users/nabeel/Documents/GitHub/Data-SUITE/src/utils/helpers.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inlier[f"{feature}_contrib"] = df_inlier["norm_interval"]


## EXAMPLE: TRAIN A DOWNSTREAM REGRESSION MODEL & SHOW RESULTS ON THE DIFFERENT TYPES OF SAMPLES

In [8]:

from src.models.benchmarks import comparison_methods

inlier_ids = inliers_dict[0]
outlier_ids = outliers_dict[0]


#####################################
#
# FIT A DOWNSTREAM MODEL ON TRAINING DATA
# MAKE PREDICTIONS ON TEST DATA
#
#####################################

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train[:, 0:-1], train[:, -1])

y_pred = regr.predict(train[:, 0:-1])


#####################################
#
# ASSESS MSE ON THE DIFFERENT TYPE OF SAMPLES IDENTIFIED
#
#####################################

mse = mean_squared_error(train[:, -1], y_pred)
print(f"MSE Train data: {mse} \n")
wandb_dict["mse_train_clean"] = mse

print("-------------------------------")

y_pred = regr.predict(test[:, 0:-1])
mse = mean_squared_error(test[:, -1], y_pred)
print(
    f"MSE Test (ALL SAMPLES - INLIERS+OUTLIERS): {mse} \n"
)
wandb_dict["mse_test_unknown"] = mse

print("-------------------------------")

y_pred = regr.predict(test[outlier_ids, 0:-1])
mse = mean_squared_error(test[outlier_ids, -1], y_pred)
print(f"MSE Outliers: {mse} \n")
wandb_dict["mse_test_outliers"] = mse

print("-------------------------------")

y_pred = regr.predict(test[inlier_ids, 0:-1])
mse = mean_squared_error(test[inlier_ids, -1], y_pred)
print(f"MSE Inliers: {mse } \n")
wandb_dict["mse_test_inliers"] = mse

y_pred = regr.predict(test[small_ci_ids, 0:-1])
mse = mean_squared_error(test[small_ci_ids, -1], y_pred)
print(f"MSE Inliers w/ SMALL CIs: {mse}\n ")
wandb_dict["mse_test_inliers_small_ci"] = mse

y_pred = regr.predict(test[large_ci_ids, 0:-1])
mse = mean_squared_error(test[large_ci_ids, -1], y_pred)
print(f"MSE Inliers w/  LARGE CIs: {mse}\n")
wandb_dict["mse_test_inliers_large_ci"] = mse

MSE Train data: 0.07331333158804787 

-------------------------------
MSE Test (ALL SAMPLES - INLIERS+OUTLIERS): 0.09230996081523919 

-------------------------------
MSE Outliers: 0.12299546905610609 

-------------------------------
MSE Inliers: 0.08819747002007144 

MSE Inliers w/ SMALL CIs: 0.044994344102423436
 
MSE Inliers w/  LARGE CIs: 0.11341212798044836



### Note the differences between samples with small CIs and Large CIs - indicating we can trust samples with small CIs more

## EXAMPLE: Compute performance metrics

In [9]:
from src.utils.uncertainty_metrics import *
from copy import deepcopy

ids = range(test.shape[0])

ids = inlier_ids

y_test_ids = noise_bool

x_train_uncert, y_train_uncert = train[:, 1:], train[:, 0]
x_test_uncert = test[:, 1:]
feature=0

df_conformal = ds.conformal_dict[feature]

df_conformal = df_conformal.iloc[ids, :]

df_conformal["pred"] = df_conformal["min"] + (df_conformal["conf_interval"] / 2)

preds = df_conformal["pred"]  # target predictions
# dc['true_val']  # ground truth observations
true = orig_test[ids, 0]
# lower bound of the prediction interval
lb = df_conformal["min"]
# upper bound of the prediction interval
ub = df_conformal["max"]

print("COMPUTING PERFORMANCE METRICS")

(
    uncert_metrics,
    excess,
    deficet,
    excess_all,
    deficet_all,
) = compute_uncertainty_metrics(
    preds=preds, lower_bound=lb, upper_bound=ub, true=true
)

idx_ordered = list(df_conformal.sort_values(by="conf_interval").index)
results, roc = [], [] 

wandb_dict = process_results(
    wandb_dict,
    results,
    roc,
    uncert_metrics,
    excess,
    deficet,
    excess_all,
    deficet_all,
    name="conformal_copula",
)

/Users/nabeel/Documents/Projects/Noise/code/venv3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


COMPUTING PERFORMANCE METRICS


## This dict could be logged - note PICP (Prediction interval coverage probability) = Coverage 

In [10]:
wandb_dict

{'mse_train_clean': 0.07331333158804787,
 'mse_test_unknown': 0.09230996081523919,
 'mse_test_outliers': 0.12299546905610609,
 'mse_test_inliers': 0.08819747002007144,
 'mse_test_inliers_small_ci': 0.044994344102423436,
 'mse_test_inliers_large_ci': 0.11341212798044836,
 'excess_conformal_copula': 1.2147045863174517,
 'deficet_conformal_copula': 0.12220596211591798,
 'excess_all_conformal_copula': 1.1896591309294628,
 'deficet_all_conformal_copula': 0.0025197105590910925,
 'roc_conformal_copula': [],
 'rmse_conformal_copula': 0.36340642968528725,
 'nll_conformal_copula': 0.7119051012903136,
 'auucc_gain_conformal_copula': -0.48191836356388684,
 'picp_conformal_copula': 0.979381443298969,
 'mpiw_conformal_copula': 2.9249438198508497,
 'r2_conformal_copula': 0.9590638096195261}